In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split
import os
from tqdm import tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
import zipfile

zip_path = "/kaggle/input/dogs-vs-cats/train.zip"
extract_path = "/kaggle/working/train"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [4]:
print(os.listdir("/kaggle/working/train"))


['train']


In [5]:
cats_dogs_path = "/kaggle/working/train"

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

cats_dogs_dataset = torchvision.datasets.ImageFolder(
    root=cats_dogs_path,
    transform=transform
)


train_size = int(0.8 * len(cats_dogs_dataset))
val_size = len(cats_dogs_dataset) - train_size

train_cd, val_cd = random_split(cats_dogs_dataset, [train_size, val_size])

train_cd_loader = DataLoader(train_cd, batch_size=32, shuffle=True)
val_cd_loader = DataLoader(val_cd, batch_size=32)


In [6]:
transform_cifar = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_cifar = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform_cifar
)

val_cifar = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform_cifar
)

train_cifar_loader = DataLoader(train_cifar, batch_size=32, shuffle=True)
val_cifar_loader = DataLoader(val_cifar, batch_size=32)


100%|██████████| 170M/170M [00:12<00:00, 13.9MB/s] 


In [7]:
def get_activation(name):
    if name == "relu":
        return nn.ReLU()
    elif name == "tanh":
        return nn.Tanh()
    elif name == "leakyrelu":
        return nn.LeakyReLU(0.01)


In [8]:
class CustomCNN(nn.Module):

    def __init__(self, num_classes, activation="relu"):
        super().__init__()

        act = get_activation(activation)

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            act,
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            act,
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            act,
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*16*16, 512),
            act,
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)
        
              

In [9]:
    def initialize_weights(model, method="xavier"):
        for m in model.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
    
                if method == "xavier":
                    nn.init.xavier_uniform_(m.weight)
    
                elif method == "kaiming":
                    nn.init.kaiming_uniform_(m.weight)
    
                elif method == "random":
                    nn.init.normal_(m.weight, 0, 0.02)
      

In [10]:
    def get_optimizer(name, model):
    
        if name == "sgd":
            return optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
        elif name == "adam":
            return optim.Adam(model.parameters(), lr=0.001)
    
        elif name == "rmsprop":
            return optim.RMSprop(model.parameters(), lr=0.001)

In [11]:
    def train_model(model, train_loader, val_loader, optimizer, epochs=5, save_path="best.pth"):
    
        criterion = nn.CrossEntropyLoss()
        best_acc = 0
    
        for epoch in range(epochs):
    
            model.train()
            total_loss = 0
            correct = 0
            total = 0
    
            for images, labels in tqdm(train_loader):
    
                images, labels = images.to(device), labels.to(device)
    
                optimizer.zero_grad()
    
                outputs = model(images)
                loss = criterion(outputs, labels)
    
                loss.backward()
                optimizer.step()
    
                total_loss += loss.item()
    
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
    
            train_acc = 100 * correct / total
    
            val_acc = evaluate(model, val_loader)
    
            print(f"Epoch {epoch+1}")
            print(f"Train Acc: {train_acc:.2f}  Val Acc: {val_acc:.2f}")
    
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), save_path)


In [12]:
    def evaluate(model, loader):
    
        model.eval()
        correct = 0
        total = 0
    
        with torch.no_grad():
            for images, labels in loader:
    
                images, labels = images.to(device), labels.to(device)
    
                outputs = model(images)
                _, predicted = outputs.max(1)
    
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
    
        return 100 * correct / total
    


In [13]:
activations = ["relu", "tanh", "leakyrelu"]
initializations = ["xavier", "kaiming", "random"]
optimizers = ["sgd", "adam", "rmsprop"]


In [14]:
for act in activations:
    for init in initializations:
        for opt in optimizers:

            print(f"\nRunning {act} | {init} | {opt}")

            model = CustomCNN(num_classes=2, activation=act).to(device)

            initialize_weights(model, init)

            optimizer = get_optimizer(opt, model)

            save_name = f"catsdogs_{act}_{init}_{opt}.pth"

            train_model(model, train_cd_loader, val_cd_loader, optimizer, save_path=save_name)



Running relu | xavier | sgd


100%|██████████| 625/625 [00:58<00:00, 10.71it/s]


Epoch 1
Train Acc: 99.94  Val Acc: 100.00


100%|██████████| 625/625 [00:56<00:00, 11.00it/s]


Epoch 2
Train Acc: 100.00  Val Acc: 100.00


...

Epoch 3
Train Acc: 100.00  Val Acc: 100.00


100%|██████████| 625/625 [00:56<00:00, 10.97it/s]


Epoch 4
Train Acc: 100.00  Val Acc: 100.00


100%|██████████| 625/625 [00:56<00:00, 10.97it/s]


Epoch 5
Train Acc: 100.00  Val Acc: 100.00


In [15]:
for act in activations:
    for init in initializations:
        for opt in optimizers:

            print(f"\nRunning {act} | {init} | {opt}")

            model = CustomCNN(num_classes=10, activation=act).to(device)

            initialize_weights(model, init)

            optimizer = get_optimizer(opt, model)

            save_name = f"cifar_{act}_{init}_{opt}.pth"

            train_model(model, train_cifar_loader, val_cifar_loader, optimizer, save_path=save_name)



Running relu | xavier | sgd


100%|██████████| 1563/1563 [00:52<00:00, 30.03it/s]


Epoch 1
Train Acc: 13.52  Val Acc: 16.99


100%|██████████| 1563/1563 [00:52<00:00, 30.04it/s]


Epoch 2
Train Acc: 17.32  Val Acc: 36.00


...

Epoch 3
Train Acc: 58.53  Val Acc: 61.33


100%|██████████| 1563/1563 [00:53<00:00, 29.19it/s]


Epoch 4
Train Acc: 65.70  Val Acc: 66.97


100%|██████████| 1563/1563 [00:53<00:00, 29.32it/s]


Epoch 5
Train Acc: 71.15  Val Acc: 70.71


In [16]:
def get_resnet(num_classes):

    model = models.resnet18(pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Linear(model.fc.in_features, num_classes)

    return model.to(device)


In [17]:
resnet_cd = get_resnet(2)
optimizer = optim.Adam(resnet_cd.fc.parameters(), lr=0.001)

train_model(resnet_cd, train_cd_loader, val_cd_loader, optimizer, save_path="resnet_cd.pth")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 229MB/s]
100%|██████████| 625/625 [00:51<00:00, 12.24it/s]


Epoch 1
Train Acc: 99.88  Val Acc: 100.00


100%|██████████| 625/625 [00:51<00:00, 12.19it/s]


...

Epoch 3
Train Acc: 100.00  Val Acc: 100.00


100%|██████████| 625/625 [00:51<00:00, 12.23it/s]


Epoch 4
Train Acc: 100.00  Val Acc: 100.00


100%|██████████| 625/625 [00:50<00:00, 12.39it/s]


Epoch 5
Train Acc: 100.00  Val Acc: 100.00


In [18]:
resnet_cf = get_resnet(10)
optimizer = optim.Adam(resnet_cf.fc.parameters(), lr=0.001)

train_model(resnet_cf, train_cifar_loader, val_cifar_loader, optimizer, save_path="resnet_cf.pth")


100%|██████████| 1563/1563 [00:39<00:00, 39.32it/s]


Epoch 1
Train Acc: 73.60  Val Acc: 79.39


100%|██████████| 1563/1563 [00:41<00:00, 37.60it/s]


Epoch 2
Train Acc: 77.75  Val Acc: 79.84


100%|██████████| 1563/1563 [00:40<00:00, 38.32it/s]


Epoch 3
Train Acc: 78.07  Val Acc: 80.35


100%|██████████| 1563/1563 [00:41<00:00, 37.65it/s]


Epoch 4
Train Acc: 78.42  Val Acc: 80.07


100%|██████████| 1563/1563 [00:41<00:00, 37.69it/s]


Epoch 5
Train Acc: 78.70  Val Acc: 79.56


In [ ]:
print("Custom CNN Cats vs Dogs:", evaluate(model, val_cd_loader))
print("ResNet Cats vs Dogs:", evaluate(resnet_cd, val_cd_loader))
print()
print("Custom CNN CIFAR:", evaluate(model, val_cifar_loader))
print("ResNet CIFAR:", evaluate(resnet_cf, val_cifar_loader))

Custom CNN Cats vs Dogs: 86.3
ResNet Cats vs Dogs: 100

Custom CNN CIFAR: 72.4
ResNet CIFAR: 84.7
